In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def get_parameters(net):
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

In [ ]:
def prune_model(net, prate):
    parameters = get_parameters(net)
    pruned_params = [None] * len(parameters)
    for idx, param in enumerate(parameters):
        if param.ndim > 1:
            flat_weights = np.abs(param).flatten()
            k = int(prate * flat_weights.size)  # Number of weights to prune
            if k > 0:
                # Find the k-th smallest magnitude
                threshold = np.partition(flat_weights, k)[k]
                # Set weights below the threshold to zero
                param[np.abs(param) < threshold] = 0
        pruned_params[idx] = param
    return pruned_params

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net = Net()